In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/29 05:17:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
trip_file = "fhvhv_tripdata_2020-03.csv"
directory = "/home/ubuntu/working/spark-examples/data/{}"

trip_file_path = directory.format(trip_file)
trip_file_path

'/home/ubuntu/working/spark-examples/data/fhvhv_tripdata_2020-03.csv'

In [3]:
trip_data = spark.read.csv(f"file:///{trip_file_path}",
                           inferSchema=True, header=True)

trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [4]:
zone_file_path = directory.format("taxi+_zone_lookup.csv")

zone_data = spark.read.csv(f"{zone_file_path}",
                              inferSchema=True, header=True)

zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [5]:
query = """
SELECT zone_data.Zone, count(*) AS trips
FROM trip_data
JOIN zone_data ON trip_data.PULocationID = zone_data.LocationID
WHERE trip_data.hvfhs_license_num = 'HV0003'
GROUP BY zone_data.Zone
order by trips desc
"""

In [6]:
zone_data.createOrReplaceTempView("zone_data")
trip_data.createOrReplaceTempView("trip_data")

In [8]:
spark.sql("""
select *
from zone_data
limit 5
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [9]:
spark.sql(query).show()

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|163091|
|       East New York|134198|
|         JFK Airport|114179|
|        East Village|112017|
|      Bushwick South|110150|
|Central Harlem North|108070|
|   LaGuardia Airport|104119|
|Washington Height...| 97324|
|Flatbush/Ditmas Park| 95724|
|            Canarsie| 94484|
|TriBeCa/Civic Center| 94155|
|             Astoria| 92676|
|             Bedford| 90352|
|      Midtown Center| 90261|
|  Stuyvesant Heights| 88749|
|            Union Sq| 88372|
|Times Sq/Theatre ...| 86870|
|Prospect-Lefferts...| 84347|
|         Brownsville| 82764|
|Mott Haven/Port M...| 82396|
+--------------------+------+
only showing top 20 rows



In [11]:
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Sort ['trips DESC NULLS LAST], true
+- 'Aggregate ['zone_data.Zone], ['zone_data.Zone, 'count(1) AS trips#206]
   +- 'Filter ('trip_data.hvfhs_license_num = HV0003)
      +- 'Join Inner, ('trip_data.PULocationID = 'zone_data.LocationID)
         :- 'UnresolvedRelation [trip_data], [], false
         +- 'UnresolvedRelation [zone_data], [], false

== Analyzed Logical Plan ==
Zone: string, trips: bigint
Sort [trips#206L DESC NULLS LAST], true
+- Aggregate [Zone#87], [Zone#87, count(1) AS trips#206L]
   +- Filter (hvfhs_license_num#17 = HV0003)
      +- Join Inner, (PULocationID#21 = LocationID#85)
         :- SubqueryAlias trip_data
         :  +- View (`trip_data`, [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23])
         :     +- Relation [hvfhs_license_num#17,dispatching_base_num#18,pickup_datetime#19,dropoff_datetime#20,PULocationID#21,DOLocationID#22,SR_Flag#23] csv
         +-

In [12]:
spark.stop()